Congressional Trading Timing Strategy

In [55]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

In [56]:
# Importing congressional trading dataset
data = pd.read_csv("congress-trading-all.csv")

# It was found later on that the values for Ticker "BNB.A" was not consistent across data sources
data['Ticker'] = data['Ticker'].str.replace(r'\.', '', regex=True)

print(data.head())


  Ticker TickerType                                   Company      Traded  \
0   BRKB         ST  BERKSHIRE HATHAWAY INC. NEW COMMON STOCK  2024-03-28   
1    NTR      Stock                               Nutrien Ltd  2024-03-28   
2    MOS      Stock                            Mosaic Company  2024-03-28   
3      D      Stock                       Dominion Energy Inc  2024-03-28   
4   SOFI      Stock                     Sofi Technologies Inc  2024-03-21   

   Transaction     Trade_Size_USD Status                  Subholding  \
0         Sale  $15,001 - $50,000    NEW   VANGUARD JOINT RETIREMENT   
1     Purchase   $1,001 - $15,000    New                         NaN   
2  Sale (Full)   $1,001 - $15,000    New                         NaN   
3     Purchase   $1,001 - $15,000    New                         NaN   
4  Sale (Full)   $1,001 - $15,000    New                         NaN   

  Description               Name       Filed Party District Chamber Comments  \
0         NaN       Kath

In [57]:
# Merging on stock tickers to only retrieve congressional data relevant to stock trading
ticker_data = pd.read_csv("stock_tickers.csv")
combined_data = pd.merge(ticker_data, data, how="inner", left_on='Symbol', right_on='Ticker')

# Check to see what these symbols that were not a part of the merged data are denoted as
excluded_tickers = data[~data['Ticker'].isin(ticker_data['Symbol'])]
unique_counts = excluded_tickers.groupby('TickerType')['Ticker'].nunique()
unique_counts_sorted = unique_counts.sort_values(ascending=False)

print(unique_counts)


TickerType
AB                    11
CS                    14
Corporate Bond         4
Cryptocurrency         3
ET                     7
GS                    42
HN                     6
OI                     2
OL                     2
OP                     7
OT                    51
Other Securities      97
PS                     1
ST                  1094
Stock                416
Stock Option           5
Name: Ticker, dtype: int64


In [62]:
# Exclude any data which were listed as non-stocks 
combined_data = combined_data[(combined_data['TickerType'] == 'ST') | (combined_data['TickerType'] == 'Stock') | (combined_data['TickerType'] == 'Stock Option')]
combined_data_types = combined_data.groupby('TickerType')['Symbol'].count()

print(combined_data_types)

TickerType
ST              19295
Stock            6226
Stock Option      404
Name: Symbol, dtype: int64


After further investigation, many of the tickers from the congress trading data which were not joining on the stock_tickers data are ETFs, which are not a part of this analysis. We also choose to exclude any tickers which were able to join on the stock_tickers data, but were listed as a non-stock ticker type in the congress trading data.
